Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, make_scorer
import torch
import transformers
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
import lightgbm as lgbm
import catboost as catboost




[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Импортируем датасет и посмотрим на него.

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


2 колонки и 159571 строчка. Text содержит текст комментария, а toxic содержит целевой признак - токсичный комментарий или нет. Сначала проверим на базовые ошибки, а потом потребуется лемматизация и подготовка к обучению моделей.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [5]:
df.isnull().sum()

text     0
toxic    0
dtype: int64

Пропусков нет.

In [6]:
df.duplicated().sum()

0

Дубликатов нет.

In [7]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Подготовим данные к обучению.

Выделим корпус данных для обучения модели из обучающей выборки.

In [8]:
corpus = list(df['text'])
corpus[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

Напишем формулу очистки текста.

In [9]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return text

Очистим от ненужных символов наш корпус.

In [10]:
for i in range(len(corpus)):
    corpus[i] = clear_text(corpus[i])
    
print(corpus[0])

Explanation Why the edits made under my username Hardcore Metallica Fan were reverted  They weren t vandalisms  just closure on some GAs after I voted at New York Dolls FAC  And please don t remove the template from the talk page since I m retired now             


Разобьем твиты на слова.

In [11]:
for i in range(len(corpus)):
    corpus[i] = nltk.word_tokenize(corpus[i])
    
print(corpus[0])

['Explanation', 'Why', 'the', 'edits', 'made', 'under', 'my', 'username', 'Hardcore', 'Metallica', 'Fan', 'were', 'reverted', 'They', 'weren', 't', 'vandalisms', 'just', 'closure', 'on', 'some', 'GAs', 'after', 'I', 'voted', 'at', 'New', 'York', 'Dolls', 'FAC', 'And', 'please', 'don', 't', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'I', 'm', 'retired', 'now']


Напишем формулу лемматизации.

In [12]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemm_list = lemmatizer.lemmatize(text)
    return lemm_list

Так как лемматизатор воспринимает только по одному слову, то напишем цикл для него.

In [13]:
for i in range(len(corpus)):
    for n in range(len(corpus[i])):
        corpus[i][n] = lemmatize(corpus[i][n])        

print(corpus[0])

['Explanation', 'Why', 'the', 'edits', 'made', 'under', 'my', 'username', 'Hardcore', 'Metallica', 'Fan', 'were', 'reverted', 'They', 'weren', 't', 'vandalism', 'just', 'closure', 'on', 'some', 'GAs', 'after', 'I', 'voted', 'at', 'New', 'York', 'Dolls', 'FAC', 'And', 'please', 'don', 't', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'I', 'm', 'retired', 'now']


Объединим твиты обратно.

In [14]:
for i in range(len(corpus)):
    corpus[i] = " ".join(corpus[i])
    
print(corpus[0])

Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren t vandalism just closure on some GAs after I voted at New York Dolls FAC And please don t remove the template from the talk page since I m retired now


Скачаем список английских стоп-слов и добавим счетчик, который преобразует корпус текстов в мешок слов.

In [15]:
nltk.download('stopwords') 
stop_words = set(stopwords.words('english')) 
count_vect = CountVectorizer() 

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Передадим счётчику корпус текстов. Для этого вызовем fit_transform(). 

In [16]:
bow = count_vect.fit_transform(corpus) 

Посмотрим на размер мешка без учета стоп-слов.

In [17]:
print("Размер мешка без учёта стоп-слов:", bow.shape)

Размер мешка без учёта стоп-слов: (159571, 164412)


Теперь передадим счетику список стоп-слов и получим новый мешок.

In [18]:
count_vect = CountVectorizer(stop_words=stop_words)
bow = count_vect.fit_transform(corpus) 

In [19]:
print("Размер мешка с учётом стоп-слов:", bow.shape)

Размер мешка с учётом стоп-слов: (159571, 164267)


Теперь рассчитаем TF-IDF.

In [20]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words) 
tf_idf = count_tf_idf.fit_transform(corpus)

In [21]:
print("Размер матрицы:", tf_idf.shape)

Размер матрицы: (159571, 164267)


Разделим данные на выборки - обучающую, валдицаионную и тестовую - и проверим все ли верно.

In [22]:
features, features_test, target, target_test = train_test_split(tf_idf, 
                                    df.toxic, test_size = 0.2, train_size = 0.8)
features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                    target, test_size = 0.25, train_size = 0.75)

In [23]:
features_test

<31915x164267 sparse matrix of type '<class 'numpy.float64'>'
	with 877422 stored elements in Compressed Sparse Row format>

In [24]:
features_train

<95742x164267 sparse matrix of type '<class 'numpy.float64'>'
	with 2600809 stored elements in Compressed Sparse Row format>

In [25]:
features_valid

<31914x164267 sparse matrix of type '<class 'numpy.float64'>'
	with 865670 stored elements in Compressed Sparse Row format>

In [26]:
target_test

102734    0
98715     1
87972     0
120570    1
131827    0
         ..
118341    0
23051     0
69805     0
154423    0
74794     0
Name: toxic, Length: 31915, dtype: int64

In [27]:
target_train

51186     0
140687    0
121290    0
115830    0
46195     0
         ..
4247      0
65338     0
52207     0
126801    0
15886     0
Name: toxic, Length: 95742, dtype: int64

In [28]:
target_valid

157495    0
20796     0
90357     0
153253    0
26535     1
         ..
71124     1
138450    0
114563    0
49673     1
149058    0
Name: toxic, Length: 31914, dtype: int64

# 2. Обучение

Начнем с модели логистической регрессии.

In [29]:
model = LogisticRegression(random_state=12345)

Попробуем подобрать оптимальные параметры с помощью gridsearch.

In [30]:
f1 = make_scorer(f1_score , average='macro')

In [31]:
parametrs_lr = { 'C': range (1, 10, 1),
              'class_weight': ['balanced'],
              }

grid = GridSearchCV(model, parametrs_lr, scoring = 'f1', cv=5)
grid.fit(features_train, target_train)
grid.best_params_

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


{'C': 9, 'class_weight': 'balanced'}

In [32]:
model = LogisticRegression(random_state=12345, C=9, class_weight='balanced')
model.fit(features_train, target_train) 

LogisticRegression(C=9, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

Сделаем предсказания и получим accuracy_score.

In [33]:
pred = model.predict(features_test)
print('Logistic Regression F1_score', f1_score(target_test, pred))
print('Logistic Regression accuracy_score', accuracy_score(target_test, pred))

Logistic Regression F1_score 0.7646885149099498
Logistic Regression accuracy_score 0.9500548331505562


Посчитаем f1.

In [34]:
print('F1_score', f1_score(target_test, pred))

F1_score 0.7646885149099498


Нижний порог в 0.75 преодолен, качество модели нас устраивает.

Перейдем к градиентному бустингу. Попробуем LGB классификатор и подберем оптимальные параметры с помощью Gridsearch.

In [35]:
lgb = lgbm.LGBMClassifier()

params_lgb = {'n_estimators': range (1, 10, 2),
              'learning_rate': [0.1, 0.9],
             }
 
grid_lgb = GridSearchCV(lgb, params_lgb, scoring = 'f1', cv=5)

grid_lgb.fit(features_train, target_train)

grid_lgb.best_params_

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'learning_rate': 0.9, 'n_estimators': 1}

Обучим модель.

In [36]:
lgb = lgbm.LGBMClassifier(n_estimators = 2, learning_rate = 0.9)

lgb.fit(features_train, target_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.9, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=2, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [37]:
predictions_lgb = lgb.predict(features_test)
print('LGB F1_score', f1_score(target_test, predictions_lgb))
print('LGB accuracy_score', accuracy_score(target_test, predictions_lgb))

LGB F1_score 0.5095119933829612
LGB accuracy_score 0.9256775810747297


In [38]:
ctb = catboost.CatBoostClassifier()

params_ctb = {'n_estimators': range (1, 10, 5),
              'learning_rate': [0.1, 0.9],
             }
 
grid_ctb = GridSearchCV(ctb, params_ctb, scoring = 'f1', cv=5)

grid_ctb.fit(features_train, target_train)

grid_ctb.best_params_

0:	learn: 0.5919562	total: 3.76s	remaining: 0us
0:	learn: 0.5940476	total: 3.95s	remaining: 0us
0:	learn: 0.5943452	total: 3.95s	remaining: 0us
0:	learn: 0.5956625	total: 4.01s	remaining: 0us
0:	learn: 0.5937408	total: 4.07s	remaining: 0us
0:	learn: 0.5919562	total: 3.97s	remaining: 19.8s
1:	learn: 0.5139538	total: 7.26s	remaining: 14.5s
2:	learn: 0.4551488	total: 10.6s	remaining: 10.6s
3:	learn: 0.4088155	total: 13.8s	remaining: 6.88s
4:	learn: 0.3725016	total: 17.1s	remaining: 3.41s
5:	learn: 0.3442528	total: 20.4s	remaining: 0us
0:	learn: 0.5940476	total: 4.01s	remaining: 20.1s
1:	learn: 0.5145838	total: 7.31s	remaining: 14.6s
2:	learn: 0.4546235	total: 10.6s	remaining: 10.6s
3:	learn: 0.4076458	total: 13.9s	remaining: 6.96s
4:	learn: 0.3716072	total: 17.3s	remaining: 3.46s
5:	learn: 0.3424688	total: 20.6s	remaining: 0us
0:	learn: 0.5943452	total: 3.95s	remaining: 19.7s
1:	learn: 0.5191434	total: 7.24s	remaining: 14.5s
2:	learn: 0.4582333	total: 10.5s	remaining: 10.5s
3:	learn: 0.41

{'learning_rate': 0.9, 'n_estimators': 6}

In [39]:
ctb = catboost.CatBoostClassifier(n_estimators = 6, learning_rate = 0.9)

ctb.fit(features_train, target_train)

0:	learn: 0.2646743	total: 4.74s	remaining: 23.7s
1:	learn: 0.2317042	total: 8.73s	remaining: 17.5s
2:	learn: 0.2123041	total: 12.8s	remaining: 12.8s
3:	learn: 0.2008536	total: 16.8s	remaining: 8.42s
4:	learn: 0.1939234	total: 20.8s	remaining: 4.17s
5:	learn: 0.1886113	total: 24.7s	remaining: 0us


In [40]:
predictions_ctb = ctb.predict(features_test)
print('CatBoost F1_score', f1_score(target_test, predictions_ctb))
print('CatBoost accuracy_score', accuracy_score(target_test, predictions_ctb))

CatBoost F1_score 0.6565349544072948
CatBoost accuracy_score 0.9433495221682594


# 3. Выводы

Целью нашей работы было построить модель, наиболее качественно определяющую токсичные твиты. После обработки всех сообщений в датасете и приведения их к требуемому формату, учитывающему форму слова и их вес в предложении, мы попробовали три разные модели. Простая логистическая регрессия с подбором параметра "С" и сбалансированным весом классов показала уровень метрики F1 выше требуемых 0.75. 
Далее мы попытались воспользоваться бустингами классификации LGBM и CatBoost, но они не дали требуемого показателя метрики F1. Поэтому наш выбор - логистическая регрессия.

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны